In [2]:
import os
import sys

parent_dir = os.path.abspath(os.path.join(os.pardir))
sys.path.append(parent_dir)

from libsql_graph_db import database as db
from dotenv import load_dotenv # type: ignore

load_dotenv()
# db_url = os.getenv("LIBSQL_URL")
# auth_token = os.getenv("LIBSQL_AUTH_TOKEN")
db_url = "local.db"
auth_token = None


In [3]:
# To initialze the database and establish a connection with tursodb
db.initialize(db_url, auth_token)

In [4]:
# merge by similarity

def merge_nodes(threshold, k ,db_url, auth_token):
    db.atomic(db.pruning(threshold, k), db_url, auth_token)
    
threshold = 0.9
k = 2
merge_nodes(threshold, k, db_url, auth_token)

In [18]:
# To add a single node
def insert_single_node(db_url, auth_token, new_label, new_node, new_node_id):
    try:
        db.atomic(db.add_node(new_label, new_node, new_node_id), db_url, auth_token)
    except Exception as e:
        assert False
new_node = {'body': 'Continuos urination', 'type': ['person', 'patient']}
new_label = "Diabetes Symptoms"
new_node_id = 1

insert_single_node(db_url, auth_token, new_label, new_node, new_node_id)
    

In [17]:
# To add multiple nodes
def bulk_insert_operations(db_url, auth_token, labels, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    # bulk add and confirm
    db.atomic(db.add_nodes(bodies, labels, ids), db_url, auth_token)

new_nodes = {
    3 : {'name': 'Peri', 'age': '90'},
    4: {'name': 'Pema', 'age': '66'},
    5: {'name': "Ella", 'age': '45'}
}
new_labels = ["Retired Doctor", "Tuberculosis Patient", "Diabetes Pateint"]

bulk_insert_operations(db_url, auth_token, new_labels, new_nodes)

In [6]:
# To find for a node
def find_a_node(db_url, auth_token, node):
            return db.atomic(db.find_node(node), db_url, auth_token)


find_a_node(db_url, auth_token, 4)

{'name': 'Pema', 'age': '66', 'id': 4}

In [7]:
# For bulk upsert: Update and insert
def bulk_upsert(db_url, auth_token, labels, nodes):
    ids = []
    bodies = []
    for id, body in nodes.items():
        ids.append(id)
        bodies.append(body)

    db.atomic(db.upsert_nodes(bodies, labels, ids), db_url, auth_token)
nodes = {
    1 : {'name': 'Stanley', 'age': '30'},
    
    2 : {'name': 'Sheeran', 'type': ['singer', 'rich']}
}
labels = ["Lunch Box", "Pop Singer"]

bulk_upsert(db_url, auth_token, labels, nodes)
    

Updated


In [8]:
# For single upsert: Update and insert a node
def single_upsert(db_url, auth_token, label, body, id):
    
    db.atomic(db.upsert_node(id, label, body), db_url, auth_token)

body = {'name': 'Sheeran', 'type': ['singer', 'rich']}
label = "related as an"
id = 2
single_upsert(db_url, auth_token, label, body, id)

Updated


In [16]:
# Bulk connect and confirm
def bulk_node_connect(db_url, auth_token, labels, edges):
    sources = []
    targets = []
    attributes = []
    for src, tgts in edges.items():
        for target in tgts:
            tgt, label = target
            sources.append(src)
            targets.append(tgt)
            if label:
                attributes.append(label)
            else:
                attributes.append({})
        

    db.atomic(db.connect_many_nodes(
        sources, targets, label, attributes), db_url, auth_token)
    
edges = {
    2: [
        (3 , {'disease':'Diabetes'}),
        (1 , {'patient': 'Diabetes Symptoms'})
    ]
}
edges_labels = ["Has", "is"]

bulk_node_connect(db_url, auth_token, edges_labels, edges)
    


In [10]:
# Single connect and confirm
def single_node_connect(db_url, auth_token, source, target, label, attribute):
    db.atomic(db.connect_nodes(
        source, target,  label, attribute), db_url, auth_token)
    
source = 3
target = 1
label = "treats"
attribute = {"person": "old man", "disease": "Asthma"}

single_node_connect(db_url, auth_token, source, target, label, attribute)

In [11]:
# To remove nodes
def remove_bulk_nodes(db_url, auth_token, ids):
        db.atomic(db.remove_nodes(ids), db_url, auth_token)

ids = [1, 4]

remove_bulk_nodes(db_url, auth_token, ids)


In [3]:
# To remove a single node
def remove_single_node(db_url, auth_token, id):
    db.atomic(db.remove_node(id), db_url, auth_token)

remove_single_node(db_url, auth_token, 3)

In [19]:
# To traverse the nodes
def traverse_nodes(db_url, auth_token, src, tgt):
    return db.traverse(db_url, auth_token, src, tgt)
traverse_nodes(db_url, auth_token, 1, 3)

['1']

In [20]:
# To generate a query clause and find nodes
kv_name_like = db._generate_clause('name', predicate='LIKE')
print(db.atomic(db.find_nodes([kv_name_like], ('Pe%',)), db_url, auth_token))


[{'name': 'Peri', 'age': '90', 'id': 3}, {'name': 'Pema', 'age': '66', 'id': 4}]


In [22]:
# Visualization
from libsql_graph_db import visualizers
raw_path = "temp_path" +  "sample1.dot"
visualizers.graphviz_visualize(db_url, auth_token, raw_path, ['2'])

In [23]:
# Visualization with bodies

raw_path = "temp_path" +  "bodies.dot"
path_with_bodies = db.traverse(db_url, auth_token, 2, with_bodies=True)
visualizers.graphviz_visualize_bodies(raw_path, path_with_bodies)

In [ ]:
# vector search node
def vector_search_node(db_url, auth_token, data, k):
    try:
        db.atomic(db.vector_search_node(data, k), db_url, auth_token)
    except Exception as e:
        assert False
        
data = {"patient": "has diabetes"}
k = 1
vector_search_node(db_url, auth_token, data, k)
    

In [4]:
# vector search edge
def vector_search_edge(db_url, auth_token, data, k):
    try:
        db.atomic(db.vector_search_edge(data, k), db_url, auth_token)
    except Exception as e:
        assert False
        
data = {'subject': 'MES', 'type': ['person', 'Dr']}
k = 1
vector_search_edge(db_url, auth_token, data, k)
    

: 